In [1]:
import numpy as np

two = np.array([
                [0, 0, 0, 0, 0, 0, 0],
                [0,+1,+1,+1,+1,+1, 0],
                [0, 0, 0, 0, 0,+1, 0],
                [0, 0, 0, 0, 0,+1, 0],
                [0,+1,+1,+1,+1,+1, 0],  
                [0,+1, 0, 0, 0, 0, 0],
                [0,+1, 0, 0, 0, 0, 0],
                [0,+1,+1,+1,+1,+1, 0],
                [0, 0, 0, 0, 0, 0, 0]])

nine=np.array([ 
                [0, 0, 0, 0, 0, 0, 0],
                [0,+1,+1,+1,+1,+1, 0],
                [0,+1, 0, 0, 0,+1, 0],
                [0,+1, 0, 0, 0,+1, 0],
                [0,+1,+1,+1,+1,+1, 0],
                [0, 0, 0, 0, 0,+1, 0],
                [0, 0, 0, 0, 0,+1, 0],
                [0, 0, 0, 0, 0,+1, 0],
                [0, 0, 0, 0, 0, 0, 0]
                ])

two = [2 * i - 1 for i in two]
nine = [2 * i - 1 for i in nine]
data = np.stack([two, nine])
# 次元数
dim = 63
# 記憶させたいパターン数
k = 2


print(w)




NameError: name 'w' is not defined

In [ ]:
""" 重み行列Wを計算 """
def fit(self, dim, data):
    w = np.zeros([dim, dim])
    print(w.shape)
    for i in data:
        w += np.outer(i, i.T)
    for i in range(dim):
        # 対角成分を0にする
        w[i][i] = 0
    return w

In [ ]:
'''
エネルギー関数E(x)を定義する
'''
def energy(self, data, w):
    e = -0.5 * np.dot(w, np.dot(data, data.T))


In [ ]:
""" データにノイズを加える """
def noise(self. noise):
    